In [1]:
print("Ok")

Ok


In [2]:
%pwd

'd:\\EPICS2.0\\AshaAI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\EPICS2.0\\AshaAI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [7]:
extracted_data=load_pdf_file(data='Data/')

In [ ]:
# extracted_data

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Lengath of Text Chunks", len(text_chunks))

Lengath of Text Chunks 5860


In [10]:
# text_chunks


In [14]:
from langchain_huggingface import HuggingFaceEmbeddings

In [15]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [16]:
embeddings = download_hugging_face_embeddings()

In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [18]:
# query_result

In [54]:
from dotenv import load_dotenv
load_dotenv()

True

In [55]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
TOGETHER_API_KEY=os.environ.get('TOGETHER_API_KEY')

In [21]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "ashaai"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "ashaai",
    "metric": "cosine",
    "host": "ashaai-4345at0.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [56]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["TOGETHER_API_KEY"] = TOGETHER_API_KEY

In [23]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [24]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [25]:
docsearch


In [26]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [29]:
retrieved_docs = retriever.invoke("what is Acromegaly?")


In [57]:
retrieved_docs

[Document(id='d42a5ba4-872b-4788-b973-0e4da5ed428f', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 45.0, 'page_label': '46', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='Whitehouse Station, NJ: Merck Research Laboratories,\n1997.\nLarsen, D. E., ed. Mayo Clinic Family Health Book.New York:\nWilliam Morrow and Co., Inc., 1996.\nJohn T. Lohr, PhD\nAcromegaly and gigantism\nDefinition\nAcromegaly is a disorder in which the abnormal\nrelease of a particular chemical from the pituitary gland\nin the brain causes increased growth in bone and soft tis-\nsue, as well as a variety of other disturbances throughout\nthe body. This chemical released from the pituitary gland'),
 Document(id='3869228d-d410-4053-85bb-296515189ae5', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 46.0,

In [68]:
from langchain.llms import Together

llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.1",  # ✅ Working model
    temperature=0.4,
    max_tokens=500
)


In [72]:
from langchain.llms import Together
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable
import os

# Set Together API key
os.environ["TOGETHER_API_KEY"] = "8c2f5d82e8f6608273adbe1cb4c151589073baf128eccf959c1bb8936b88d92b"  # Replace with your actual API key

# Initialize Together LLM
llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.1",
    temperature=0.4,
    max_tokens=500
)

# System prompt template
system_prompt = (
    "You are AshaAI, a helpful and concise medical assistant designed for Indian farmers. "
    "Use the following retrieved context from trusted medical sources to answer the user's query. "
    "If the answer is unknown, say so honestly. Keep the answer short (2-3 sentences), friendly, and informative.\n\n"
    "{context}"
)

# Define the ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [73]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [75]:
response = rag_chain.invoke({"input": "what is Acromegaly?"})
print(response["answer"])



Acromegaly is caused by a tumor of the pituitary gland.
The pituitary gland is a small gland located in the base of the
brain. It controls the function of many other glands in the
body, including the thyroid, parathyroid, and adrenal glands.
The pituitary gland also produces a hormone called growth
hormone (GH). When the pituitary gland produces too much
GH, it causes a disorder called acromegaly.

The symptoms of acromegaly are caused by the abnormal
release of a particular chemical (insulin-like growth factor
(IGF-1)) from the pituitary gland. IGF-1 stimulates the
growth of bones and soft tissues, which leads to the
symptoms of acromegaly.

Symptoms of acromegaly include:

1. Enlarged hands and feet
2. Deepening of the voice
3. Thickening of the skin
4. Sleep apnea
5. Joint pain and stiffness
6. Increased sweating
7. Fatigue
8. Decreased sexual function
9. Irritability
10. Depression

Acromegaly can also cause other problems, such as high blood pressure, diabetes, and heart disease.

In [78]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])



AshaAI: Stats is short for statistics. Statistics is a branch of mathematics that deals with the collection, analysis, interpretation, presentation, and organization of data. It is used to describe patterns, trends, and relationships in data. In sports, stats are used to track and analyze the performance of athletes and teams.
